In [66]:
#Import necessary libraries

import numpy as np ## numerical calculations
import matplotlib.pyplot as plt # plotting
import cartopy # mapping
import astropy # astonomy related conversions
import math # 
import pandas as pd
from astropy.time import Time,TimeDelta

In [67]:
# private coordinates for home
lat=39.921653
lon=32.912377

# a is the semi major axis of the orbit. it was given in km in the assignment but we needed the conversion (km to meter)
a= 7171800 # in meter

# e is the eccentricity of the orbit
e = 0.02 # unitless

# i is the inclination of the orbit
i = 60 # in degrees

# RAAN is longitude of the ascending node (right ascension of the ascending node) 
RAAN = 45.916666667 # in degrees --> 3h + 03m / 60 + 40s / 3600

# w is Argument of perigee
w = 10 # in degrees

true_anomaly = 120 # in degrees

In [68]:
## Define elementary rotation matrices and reflection matrices



"""
Rotation about x axis
"""
def getR1(alpha):
    return np.matrix(
        [
            [1.0,0,0],
            [0,np.cos(alpha),np.sin(alpha)],
            [0,-np.sin(alpha),np.cos(alpha)],
        ]
    )


"""
Rotation about y axis
"""
def getR2(alpha):
    return np.matrix(
        [
            [np.cos(alpha),0, -np.sin(alpha)],
            [0,1.0,0],
            [np.sin(alpha),0, np.cos(alpha)],
        ]
    )

"""
Rotation about z axis
"""
def getR3(alpha):
    return np.matrix(
        [
            [np.cos(alpha),np.sin(alpha),0],
            [-np.sin(alpha),np.cos(alpha),0],
            [0,0,1],

        ]
    )



"""
Reflection about x
"""

def getS1():
    return np.matrix(
        [
            [-1,0,0],
            [ 0,1,0],
            [ 0,0,1],

        ]
    )

"""
Reflection about y
"""
def getS2():
    return np.matrix(
        [
            [ 1,0,0],
            [ 0,-1,0],
            [ 0,0,1],

        ]
    )


"""
Reflection about z
"""
def getS3():
    return np.matrix(
        [
            [ 1,0,0],
            [ 0,1,0],
            [ 0,0,-1],

        ]
    )

In [69]:
## Lets create the observer class

class SphericalEarthModel():
    R = 6371000


class Observer(object):

    def __init__(self, name, lat, lon, height):
        self.name = name
        self.lat = lat
        self.lon = lon
        
        self.rlat = np.radians(lat)
        self.rlon = np.radians(lon)
        
        self.height = height
        
        # geocentric position vector
        self.r_L = np.array(
            [
                (SphericalEarthModel.R+self.height)*np.cos(self.rlat)*np.cos(self.rlon),
                (SphericalEarthModel.R+self.height)*np.cos(self.rlat)*np.sin(self.rlon),
                (SphericalEarthModel.R+self.height)*np.sin(self.rlat),
            ]
        ).reshape((-1,1)) ## as a column vector
        
        self.R_L_to_E = getR3(np.pi-self.rlon)*getR2(np.pi/2-self.rlat)*getS2()
        
        RltoE = getR3(-(np.pi/2+self.rlon))*getR2(-(np.pi/2-self.rlat))
        
        print ("All Equals topocentric to geocentric , :")
        print (self.R_L_to_E)
        print (RltoE)
    
    
    def az_el_to_x(self, azimuth, elevation, dist):
        a = np.radians(azimuth)
        e = np.radians(elevation)
        
        x = np.array([
            dist*np.cos(e)*np.cos(a),
            dist*np.cos(e)*np.sin(a),
            dist*np.sin(e)
        ]).reshape((-1,1)) ## as a column vector
        
        return x
    
    
    def x_to_az_el(self, x):
        
        dist = np.sqrt(x[0,0]**2 + x[1,0]**2+ x[2,0]**2) # sqrt (x^2 + y^2 + z^2)
        
        if (np.allclose(dist,0)):
            print ("Distance is zero !!!! in azimuth conversion")
            
        x = x/dist ## unit vector
        
        zenith = np.arccos(x[2,0])
        elev = np.pi/2 - zenith
        
        azim = np.arctan2(x[1,0],x[0,0])
        
        if (azim <0):
            azim += 2*np.pi
        
        return np.degrees(azim),np.degrees(elev),dist
    
    def convert_to_ECEF(self, x):
        ## rotate direction vector x to ECEF axes
        ## add observer location as a shift
        print (self.R_L_to_E*x.reshape((-1,1)))
        return self.r_L + self.R_L_to_E*x.reshape((-1, 1))
    
    
    def convert_fromECEF(self,X):
        ## subtract observer location to obtain direction vector in ECEF
        ## rotate direction vector to local
        Dx  = X - self.r_L
                
        return self.R_L_to_E.T*Dx
    
    
    def convert_to_lat_lon(self,x):
        dist = np.sqrt(x[0,0]**2 + x[1,0]**2+ x[2,0]**2) # sqrt (x^2 + y^2 + z^2)
        
        if (np.allclose(dist,0)):
            print ("Distance is zero !!!! in azimuth conversion")
                
        colat = np.arccos(x[2,0]/dist)
        lat = np.pi/2 - colat
        
        lon = np.arctan2(x[1,0],x[0,0])
        
        if (lon <0):
            lon += 2*np.pi
        
        return np.degrees(lat),np.degrees(lon),dist
    
    def __str__(self):
        return "Observer at (%3.4f,%3.3f, %6.2f)" % (self.lat,self.lon,self.height)

In [70]:
class Satellite(object):
    
    GM = 3986004.418e8
    
    def __init__(self,a,e,i,ran,w,T0):
        self.a = a # semi major axis in meters
        self.i = i # inclination in degrees
        self.e = e # eccentricity
        self.ran = ran # Right ascension of the ascending node in degrees
        self.w = w # argument of perigee in degrees
        self.T0 = T0 # perigee passage time
        self.T = 2*np.pi*np.sqrt(self.a**3/Satellite.GM)
        self.n = np.sqrt(Satellite.GM/self.a**3)
        
        
        self.R_o_to_I = getR3(-np.radians(self.ran))*getR1(-np.radians(self.i))*getR3(-np.radians(self.w))
        
        
    
    """
    given time t, return satellite position
    """
    def pos(self, t):
        
        dt = (t.tt-self.T0).sec ## Use terrestrial time for time difference
        
#        dt = np.mod(dt,self.T)
        
        M = self.n*(dt)
        
        E = M
        E_new = E
        
        i =0 ## iteration
        while(True and i<5):
            E_new = M+self.e*np.sin(E)
            
            if (np.abs(E_new-E)<1e-10):
                E = E_new
                ## Early break with precision
                break
                
            E = E_new
            i = i +1
        
        sv = np.sqrt(1-self.e**2)*np.sin(E)
        cv = np.cos(E)-self.e
        
        v = np.arctan2(sv,cv)
        
        if (v<0):
            v+=2*np.pi
        
        r = self.a*(1-self.e**2)/(1+self.e*np.cos(v))
        
        r_o = np.array([
            r*np.cos(v),
            r*np.sin(v),
            0
        ]).reshape((-1,1))
        
        r_i = self.R_o_to_I*r_o
        
        gast = t.sidereal_time("mean",longitude=0).rad
        
        R_gast = getR3(gast)
        
#        ECI -> ECEF
        r_e = R_gast*r_i
        
        return r_i, r_e

# Q1 ) ECI and ECEF / Azimuth and Elevation

Q1)

ECI is defined as an Earth-centered and fixed coordinate system. This system is defined as Earth-centered and earth-fixed in time, and is used to express the positions and movements of satellites or spacecraft.

ECEF (Earth Centered Earth Fixed) is defined as an Earth-centered and world-fixed coordinate system. This system is defined in an Earth-centered and earth-fixed time period and is used to express the positions and movements of objects on Earth.

Conversions between the ECI and ECEF coordinate systems are necessary because ECEF coordinates are used for the satellite positioning system, while ECI coordinates are used for the orbital motion of the spacecraft.

In this python code, ECI coordinates were found by using the relevant "Transformation matrices", "Reflection matrices" and "Spacecraft elements", then ECEF coordinates were obtained by using ECI.

Azimuth and elevation were found using classical Kepler elements.

"COE" given for choice a and b in the homework and "the time given for T0 was used". Also, our coordinates were found in the WGS84 system and results were found relative to our own house.

In [71]:
# Q1) a ) ECI and ECEF  / b ) azimuth and elevation

# For T0 (Time of the perigee passage), the date and time given in the assignment are entered.
T0 = Time("2012-02-02T13:15:00", format='isot', scale='tt')

home = Observer("Home", lat, lon,1000)


geos = Satellite(a, e, i, RAAN, w, T0)


pos = geos.pos(T0)
x = home.convert_fromECEF(pos[1])
azel = home.x_to_az_el(x)


print('ECEF:', pos[0])
print('ECI:', pos[1])
print ("Azimuth and Elevation : ",azel[0],azel[1])

All Equals topocentric to geocentric , :
[[-0.53874193 -0.54335581  0.64383352]
 [-0.34869289  0.83950251  0.4167119 ]
 [ 0.76692268  0.          0.64173951]]
[[-0.34869289 -0.83950251 -0.4167119 ]
 [ 0.53874193 -0.54335581  0.64383352]
 [-0.76692268  0.          0.64173951]]
ECEF: [[4377028.47339589]
 [5396515.12923081]
 [1056951.61647098]]
ECI: [[1166151.38999446]
 [6849879.17649278]
 [1056951.61647098]]
Azimuth and Elevation :  113.3235386860036 -20.54618330184449


# Q2) specific potential energy, specific kinetic energy and total mechanical energy (all in mega joules per kg)

Q2) Discuss where the potential energy and kinetic energy is minimum and maximum?

The potential energy varies depending on the distance of the satellite from the center of the Earth. At the apogee point, the potential energy is greatest because the satellite is farthest from the center of the Earth. At Perigee, however, the potential energy is lowest, as the satellite's distance from the Earth's center is closest.

Kinetic energy changes depending on the speed of the satellite. At the apogee point, the satellite's velocity is lowest, so its kinetic energy is also lowest. At the perigee point, however, the satellite's velocity is highest and its kinetic energy is highest.

Mechanical Energy is Conserved. The total mechanical energy, the sum of kinetic and potential energy, is constant in a conservative field.

Note : In this python code, the potential, kinetic and total mechanical energies of the spacecraft are calculated using 1 Degree increments. In addition, attention was paid to units. The results were found as "Mega Joule per kg".

In [72]:
# Q2) calculate specific potential energy, specific kinetic energy and total mechanical energy (all in mega joules per kg) 

import pandas as pd

# necessary classical orbit elements
mass = 1 # in kg 
eccentricity = 0.02 # unitless
semimajor_axis = 7171.8 # in km
true_anomaly = 120 # in degree
mu = 3.986*(10**5) #  gravitational parameter (km3/s2) = 3.986 × 105 km^3/s^2 for Earth

def distance():

    dist = []
    for i in range(true_anomaly, 360+1):
        R = (semimajor_axis*(1-eccentricity**2))/(1+ eccentricity*math.cos(i)) # magnitude of spacecraft’s position vector(km)
        dist.append(R)
    
    return dist

def potential_energy():
    
    R = dist()
    potential = []
    for i in R:
        pe = ((-((mass*mu)/i))*(1e-3)) # spacecraft’s potential energy (as mega Joules per kg)
        potential.append(pe)
    
    return potential
        
def kinetic_energy():
    
    R = dist()
    kinetic = []
    for i in R:
        V = ((2*((mu/i)+(-mu/(2*semimajor_axis))))**(1/2)) # spacecraft’s velocity (km/s)
        ke = ((1/2)*(mass)*(V**2))*(1e-3) # kinetic energy (as mega Joules per kg)
        kinetic.append(ke)
    
    return kinetic


spacecraft_pos = distance()
p_energy = potential_energy()
k_energy = kinetic_energy()

df = pd.DataFrame()
df['R (Distance)'] = spacecraft_pos
df['Potential_E'] = p_energy
df['Kinetic_E'] = k_energy
df['T_Mechanical_E'] = df['Potential_E'] + df['Kinetic_E'] # Total Mechanical energy (as mega Joules per kg)
print(df)

     R (Distance)  Potential_E  Kinetic_E  T_Mechanical_E
0     7054.065561    -0.056506   0.028717       -0.027789
1     7175.915399    -0.055547   0.027758       -0.027789
2     7295.399524    -0.054637   0.026848       -0.027789
3     7298.548971    -0.054614   0.026824       -0.027789
4     7182.258133    -0.055498   0.027708       -0.027789
..            ...          ...        ...             ...
236   7247.241631    -0.055000   0.027211       -0.027789
237   7109.753355    -0.056064   0.028274       -0.027789
238   7029.742829    -0.056702   0.028913       -0.027789
239   7076.425304    -0.056328   0.028538       -0.027789
240   7209.838620    -0.055286   0.027496       -0.027789

[241 rows x 4 columns]


In [73]:
# Q2) Special value in 120 degrees true anomaly

# necessary classical orbit elements
mass = 1 # in kg 
eccentricity = 0.02 # unitless
semimajor_axis = 7171.8 # in km
true_anomaly = 120 # in degree
mu = 3.986*(10**5) 

# magnitude of the spacecraft’s position vector (km)
R = (semimajor_axis*(1-eccentricity**2))/(1+ eccentricity*math.cos(true_anomaly))

# Specific Potential Energy
potential_e = ((-((mass*mu)/R))*(1e-3)) # spacecraft’s potential energy (as mega Joules per kg)

# Specific Kinetic Energy
V = ((2*((mu/R)+(-mu/(2*semimajor_axis))))**(1/2)) # spacecraft’s velocity (km/s)
kinetic_e = ((1/2)*(mass)*(V**2))*(1e-3) # kinetic energy (as mega Joules per kg)

# Total Mechanical Energy (as mega Joules per kg)
t_mechanical_e = potential_e + kinetic_e

print("Specific (True Anomaly in 120 degree) Potential Energy:", potential_e)
print("Specific (True Anomaly in 120 degree) Kinetic Energy:", kinetic_e)
print("Total Mechanical Energy:", t_mechanical_e)

Specific (True Anomaly in 120 degree) Potential Energy: -0.056506421232850434
Specific (True Anomaly in 120 degree) Kinetic Energy: 0.028717023870960815
Total Mechanical Energy: -0.02778939736188962
